## Libraries & settings

In [6]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver
!pip install beautifulsoup4

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
#chrome_options.add_argument('useAutomationExtention', False)
chrome_options.add_argument('--ifnore-certificate-errors')
chrome_options.add_argument('--start-maximized')

wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

     |████████████████████████████████| 954 kB 4.6 MB/s 
     |████████████████████████████████| 138 kB 53.9 MB/s 
     |████████████████████████████████| 356 kB 57.0 MB/s 
     |████████████████████████████████| 55 kB 2.9 MB/s 
     |████████████████████████████████| 3.5 MB 33.9 MB/s 
     |████████████████████████████████| 54 kB 3.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.7 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [69.5 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:13 https://developer.download.nvi

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: use options instead of chrome_options
  from ipykernel import kernelapp as app


In [7]:
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from bs4 import BeautifulSoup 
import pandas as pd 
import os
import time

# Scrapping ebay


In [ ]:
# https://www.youtube.com/watch?v=9ELd7XWD0PA&t=1067s

#user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:87.0) Gecko/20100101 Firefox/87.0'
#FireFoxDriverPath = os.path.join(os.getcwd(), 'Drivers', 'geckodriver.exe')
#FireFoxProfile = webdriver.FirefoxProfile()
#FireFoxProfile.set_preference("general.useragent.override", user_agent)
url = "https://www.ebay.com/"
browser = wd
browser.implicitly_wait(7)
browser.get(url)

search_keyword = 'Playstation 5'
search_field = browser.find_element_by_id('gh-ac')
search_field.clear()
search_field.send_keys(search_keyword)
search_field.send_keys(Keys.ENTER)

total_result = int(browser.find_element_by_class_name('srp-controls__count-heading').find_element_by_class_name('BOLD').text.replace(',', ''))
ebay_listings = []

current_page = 1
to_continue = True

In [ ]:
while to_continue:
    try:
        print('Processing page {0}'.format(current_page))

        soup = BeautifulSoup(browser.page_source, 'html.parser')
        item_list = soup.find_all('li', 's-item--watch-at-corner')
        
        for listing in item_list:
            product_detail = {}

            product_detail['product title'] = listing.h3.text
            product_detail['product url'] = listing.a['href']
            
            listing_subtitles = listing.find_all('div', 's-item__subtitle')
            if len(listing_subtitles) == 2:
                listing_subtitle_1 = listing_subtitles[0].text
                listing_subtitle_2 = listing_subtitles[1].text
            elif len(listing_subtitles) == 1:
                listing_subtitle_1 = listing_subtitles[0].text
                listing_subtitle_2 = None
            else:
                listing_subtitle_1 = None
                listing_subtitle_2 = None

            product_detail['subtitle1'] = listing_subtitle_1
            product_detail['subtitle2'] = listing_subtitle_2


            # Star rating and product review count
            try:
                x_star_rating = listing.find('div', 'x-star-rating')
                stars = len(x_star_rating.find_all('svg', 'icon--star-full-small'))
                if int(stars) > 4: print(stars) 
                review_count = int(listing.find('span', 's-item__reviews-count').span.text.replace( ' product ratings', ''))
                
                product_detail['stars'] = stars
                product_detail['product review_count'] = review_count
            except Exception as e:
                product_detail['stars'] = ''
                product_detail['product review_count'] = ''

            product_detail['price'] = listing.find('span', 's-item__price').text

            try:
                product_detail['shipping detail'] = listing.find('span', 's-item__logisticsCost').text
            except AttributeError:
                product_detail['shipping detail'] = 'Not Available'
                
            ebay_listings.append(product_detail)


        time.sleep(1)
        browser.find_element_by_xpath('//a[@class="pagination__item" and text()="{0}"]'.format(current_page+1)).click()
        current_page +=1

        if current_page > 5:
            raise Exception('Stop')

    except NoSuchElementException:
        print('Last page {0}'.format(current_page))
        to_continue = False



In [ ]:
df = pd.DataFrame(ebay_listings)
df.head()

,product title,product url,subtitle1,subtitle2,stars,product review_count,price,shipping detail
0,Sony PlayStation 5 PS5 Console Disc Version *N...,https://www.ebay.com/itm/333896636458?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,"$1,049.00",Not Available
1,PS5 Sony PlayStation 5 Console Disc Version,https://www.ebay.com/itm/324483721021?epid=190...,Pre-Owned · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,$859.99,Free shipping
2,Sony PlayStation 5 Console Disc Version (PS5) ...,https://www.ebay.com/itm/144223680923?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,$949.99,Free shipping
3,Sony PlayStation 5 PS5 Console Disc Version Bl...,https://www.ebay.com/itm/193955663603?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,$899.99,Free shipping
4,New ListingSony PlayStation 5 PS5 DISC Edition...,https://www.ebay.com/itm/203647656449?hash=ite...,Brand New · Sony PlayStation 5,None,,,$760.00,+$23.77 shipping


In [ ]:
df['amount'] = df.price.apply(lambda x: int("".join(filter(str.isdigit, x))))

In [ ]:
df.sort_values(by='amount')

,product title,product url,subtitle1,subtitle2,stars,product review_count,price,shipping detail,amount
7,Turtle Beach Elite Atlas Pro Performance Heads...,https://www.ebay.com/itm/324391510282?hash=ite...,None,None,,,$59.95,Free shipping,5995
8,"Razer Kraken Gaming Headset For PC, PS4, PS5, ...",https://www.ebay.com/itm/144123396447?hash=ite...,None,None,,,$59.99,Free shipping,5999
9,SteelSeries Arctis 5 RGB Wired Gaming Headset ...,https://www.ebay.com/itm/334129396663?hash=ite...,None,None,,,$89.95,Free shipping,8995
191,New ListingSony PlayStation 5 PS5 Blu-Ray Edit...,https://www.ebay.com/itm/324827522681?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,$102.50,+$77.61 shipping,10250
113,New Listing🎁Sony PlayStation 5 Console🎁Digital...,https://www.ebay.com/itm/265357244952?epid=250...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,461,$202.50,+$50.00 shipping,20250
...,...,...,...,...,...,...,...,...,...
129,New Listingplaystation 5 console,https://www.ebay.com/itm/304180922459?_trkparm...,Brand New · Sony PlayStation 5,None,,,"$1,124.00",+$10.00 shipping,112400
137,Sony PlayStation 5 Console PS5 [Disc Version] ...,https://www.ebay.com/itm/284478117917?epid=190...,Brand New · Sony PlayStation 5,None,0,1976,"$1,151.00",+$30.15 shipping,115100
64,Sony PlayStation 5 PS5 Console Disc Version BR...,https://www.ebay.com/itm/203581141972?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,"$1,199.95",Free shipping,119995
103,PS5 Sony Playstation 5 Console Disc Version 🔥B...,https://www.ebay.com/itm/304128005531?epid=190...,Brand New · Sony PlayStation 5 · Sony PlayStat...,None,0,1976,"$1,199.99",Free shipping,119999


In [ ]:
#df.to_excel('playstation 5 listings.xlsx', index=False)
print(df.iloc[103])

product title           PS5 Sony Playstation 5 Console Disc Version 🔥B...
product url             https://www.ebay.com/itm/304128005531?epid=190...
subtitle1               Brand New · Sony PlayStation 5 · Sony PlayStat...
subtitle2                                                            None
stars                                                                   0
product review_count                                                 1976
price                                                           $1,199.99
shipping detail                                             Free shipping
amount                                                             119999
Name: 103, dtype: object


In [ ]:
# https://towardsdatascience.com/scraping-multiple-amazon-stores-with-python-5eab811453a8

# scrapping Amazon



In [ ]:
# https://www.youtube.com/watch?v=_AeudsbKYG8

def get_url(search_term):
  """Genetare the Url from search term"""
  template = "https://www.amazon.com.mx/s?k={}&__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_1"
  search_term = search_term.replace(' ', '+')
  return template.format(search_term)

In [ ]:
url = get_url('jetson nano')
print(url)

https://www.amazon.com.mx/s?k=jetson+nano&__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_1


In [ ]:
wd.get(url)

In [ ]:
# Extract the collection
soup = BeautifulSoup(wd.page_source, 'html.parser')

In [ ]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})

In [ ]:
len(results)

60

In [ ]:
# prototype the record
item = results[0]
atag = item.h2.a
description = atag.text.strip()
url = 'https://www.amazon.com.mx' + atag.get('href')

In [ ]:
# price
price_parent = item.find('span', 'a-price')
price = price_parent.find('span', 'a-offscreen').text

In [ ]:
# stars
rating = item.i.text

In [ ]:
# find the reviews
review_count = item.find('span', {'class':'a-size-base'}).text

In [ ]:
# error handling
def extract_record(item):

  # description and url
  atag = item.h2.a
  description = atag.text.strip()
  url = 'https://www.amazon.com.mx' + atag.get('href')

  try:
    # price
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
  except AttributeError:
    return

  try:
    # reviews and rating
    rating = item.i.text
    review_count = item.find('span', {'class':'a-size-base'}).text
  except AttributeError:
    rating = ''
    review_count = ''

  result = (description, price, rating, review_count, url)

  return result


In [ ]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
  record = extract_record(item)
  if record:
    records.append(record)

In [ ]:
print(records[0])

('NVIDIA Jetson Nano - Kit de desarrollador de 2 GB', '$1,482.99', '4.4 de 5 estrellas', '272', 'https://www.amazon.com.mx/NVIDIA-Jetson-Nano-Kit-desarrollador/dp/B08J157LHH/ref=sr_1_1?__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&dchild=1&keywords=jetson+nano&qid=1634270630&sr=8-1')


In [ ]:
# getting the next page
def get_urls(search_term):
  """Genetare the Url from search term"""
  template = "https://www.amazon.com.mx/s?k={}&__mk_es_MX=%C3%85M%C3%85%C5%BD%C3%95%C3%91&ref=nb_sb_noss_1"
  search_term = search_term.replace(' ', '+')

  # add term query to url
  url = template.format(search_term)

  # add page query placeholder
  url += '&page={}'

  return template.format(search_term)

# error handling
def extract_record(item):

  # description and url
  atag = item.h2.a
  description = atag.text.strip()
  url = 'https://www.amazon.com.mx' + atag.get('href')

  try:
    # price
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
  except AttributeError:
    return

  try:
    # reviews and rating
    rating = item.i.text
    review_count = item.find('span', {'class':'a-size-base'}).text
  except AttributeError:
    rating = ''
    review_count = ''

  result = (description, price, rating, review_count, url)

  return result

def main(search_term):
  """run maun program"""
  # set webdrriver if needed

  records = []
  url = get_urls(search_term)

  for page in range(1,6):
    wd.get(url.format(page))
    soup = BeautifulSoup(wd.page_source, 'html.parser')
    results = soup.find_all('div', {'data-component-type': 's-search-result'})

    for item in results:
      record = extract_record(item)
      if record:
        records.append(record)
  
  wd.close()

  return pd.DataFrame(records, columns = ['Description', 'Price', 'Rating', 'Reviews', 'Url'])

In [ ]:
amazon_scrapping_df = main('jetson nano')

In [ ]:
amazon_scrapping_df.head(10)

,Description,Price,Rating,Reviews,Url
0,NVIDIA Jetson Nano - Kit de desarrollador de 2 GB,"$1,482.99",4.4 de 5 estrellas,272,https://www.amazon.com.mx/NVIDIA-Jetson-Nano-K...
1,Módulo de cámara para NVIDIA Jetson Nano Devel...,"$1,019.00",,,https://www.amazon.com.mx/Development-pulgadas...
2,Waveshare Jetson Nano 2GB Developer Kit Packag...,"$3,449.99",4.6 de 5 estrellas,13,https://www.amazon.com.mx/Waveshare-Jetson-Nan...
3,TOP1 Intel AC8265 - NIC de red inalámbrica de ...,$433.32,,,https://www.amazon.com.mx/TOP1-Intel-AC8265-in...
4,Yahboom Carcasa de aleación de aluminio para N...,$585.03,4.0 de 5 estrellas,14,https://www.amazon.com.mx/Yahboom-aleaci%C3%B3...
5,Geekworm N100 - Carcasa de metal para Jetson N...,$499.00,4.3 de 5 estrellas,79,https://www.amazon.com.mx/Geekworm-NVIDIA-Encl...
6,Bicool Power Supply Adapter for Jetson Nano (A...,$317.46,,,https://www.amazon.com.mx/Adaptador-alimentaci...
7,"WENDi NVIDIA Jetson Nano Developer Kit, Small ...","$4,343.46",,,https://www.amazon.com.mx/WENDi-NVIDIA-Develop...
8,Geekworm UPS T208 18650 de 6 celdas (salida má...,"$1,252.93",3.1 de 5 estrellas,3,https://www.amazon.com.mx/Geekworm-tarjeta-exp...
9,Nvidia Jetson Xavier NX Developer Kit (8126740...,"$23,956.00",4.7 de 5 estrellas,120,https://www.amazon.com.mx/nVidia-Jetson-Xavier...


# Scrapping mercado libre

In [ ]:
# Search link
# https://listado.mercadolibre.com.mx/monitor#D[A:monitor]
# Page 2 

In [ ]:
def get_url(search_term):
  link = 'https://listado.mercadolibre.com.mx/{}'.format(search_term.replace(' ','-'))
  return link

In [ ]:
print(get_url('monitor curvo'))

https://listado.mercadolibre.com.mx/monitor-curvo


In [ ]:
url = get_url('monitor curvo')
wd.get(url)
soup = BeautifulSoup(wd.page_source, 'html.parser')

In [ ]:
results = soup.find_all('li', {'class': 'ui-search-layout__item'})

In [ ]:
print(len(results))

50


In [ ]:
print(results[0])

<li class="ui-search-layout__item"><div class="ui-search-result__wrapper"><div class="andes-card andes-card--flat andes-card--default ui-search-result ui-search-result--core andes-card--padding-default"><div class="ui-search-result__image"><a class="ui-search-link" href="https://www.mercadolibre.com.mx/monitor-gamer-curvo-samsung-c32r500-led-32-dark-blue-gray-100v240v/p/MLM17952953?pdp_filters=category:MLM1656#searchVariation=MLM17952953&amp;position=1&amp;search_layout=stack&amp;type=product&amp;tracking_id=b02694ba-73e3-432e-80a7-5292855d8b8b" title='Monitor gamer curvo Samsung C32R500 led 32 " dark blue gray 100V/240V'><div class="carousel-container arrow-visible"><div class="slick-initialized slick-slider"><div class="slick-list" role="presentation"><div class="slick-track"><div class="slick-slide slick-active" data-index="0" style="outline:none" tabindex="-1"><img alt='Monitor gamer curvo Samsung C32R500 led 32 " dark blue gray 100V/240V' class="ui-search-result-image__element" he

In [ ]:
# prototype the record
item = results[0]
atag = item.h2.text.strip()
#description = atag.text.strip()
#url = 'https://listado.mercadolibre.com.mx/' + atag.get('href')

In [ ]:
print(item.a['href'])

https://www.mercadolibre.com.mx/monitor-gamer-curvo-samsung-c32r500-led-32-dark-blue-gray-100v240v/p/MLM17952953?pdp_filters=category:MLM1656#searchVariation=MLM17952953&position=1&search_layout=stack&type=product&tracking_id=b02694ba-73e3-432e-80a7-5292855d8b8b


In [ ]:
price_parent = item.find('div', {'class':'ui-search-price__second-line'})
price = price_parent.find('span', 'price-tag-amount').text
print(price)

$6,599


In [ ]:
stars = item.find_all('svg', {'class':'ui-search-icon ui-search-icon--star ui-search-icon--star-full'})
print(len(stars))

5


In [ ]:
reviews = item.find('span', {'class':'ui-search-reviews__amount'}).text
print(reviews)

6


In [24]:
# getting the next page
def get_urls(search_term):
  """Genetare the Url from search term"""
  template = "https://listado.mercadolibre.com.mx/{}"
  search_term = search_term.replace(' ', '-')

  # add term query to url
  url = template.format(search_term)

  # add page query placeholder
  url += '_Desde_{}_NoIndex_True'

  return template.format(search_term)

# error handling
def extract_record(item):

  # description and url
  description = item.h2.text.strip()
  url = item.a['href']

  try:
    # price
    price_parent = item.find('div', {'class':'ui-search-price__second-line'})
    price = price_parent.find('span', 'price-tag-amount').text
  except AttributeError:
    return ''

  try:
    # reviews and rating
    stars = len(item.find_all('svg', {'class':'ui-search-icon ui-search-icon--star ui-search-icon--star-full'}))
    reviews = item.find('span', {'class':'ui-search-reviews__amount'}).text
  except AttributeError:
    stars = ''
    reviews = ''

  result = (description, price, stars, reviews, url)

  return result

def main(search_term):
  """run maun program"""
  # set webdrriver if needed

  records = []
  url = get_urls(search_term)

  items_pp = 1
  to_continue = True
  
  while to_continue:

    try:
      wd.get(url.format(items_pp))
      soup = BeautifulSoup(wd.page_source, 'html.parser')
      results = soup.find_all('li', {'class': 'ui-search-layout__item'})

      for item in results:
        record = extract_record(item)
      
        if record:
          records.append(record)
      items_pp += 50 
    
    except NoSuchElementException:
      print('Last page {0}'.format(current_page))
      to_continue = False

    if items_pp >= 151: to_continue = False 

  wd.close()

  return pd.DataFrame(records, columns = ['Description', 'Price', 'Rating', 'Reviews', 'Url'])

In [25]:
mercado_libre_df = main('beyerdynamic')

In [26]:
mercado_libre_df.head(10)

,Description,Price,Rating,Reviews,Url
0,Audífonos Beyerdynamic DT 990 PRO black y grey,"$4,040",4,4,https://www.mercadolibre.com.mx/audifonos-beye...
1,Almohadillas Akg Beyerdynamic Pioneer Universa...,$159,,,https://articulo.mercadolibre.com.mx/MLM-84341...
2,Beyerdynamic Audifonos Dt 990 Pro 250 Ohms,"$5,299",,,https://articulo.mercadolibre.com.mx/MLM-78029...
3,Almohadillas Akg Beyerdynamic Pioneer Terciope...,$199,,,https://articulo.mercadolibre.com.mx/MLM-90995...
4,Beyerdynamic Audifonos T1 (3rd Generation),"$29,999",,,https://articulo.mercadolibre.com.mx/MLM-94366...
5,Beyerdynamic Dt 990 Edition 600 Ohm Auriculare...,"$5,560",,,https://articulo.mercadolibre.com.mx/MLM-76993...
6,Beyerdynamic Audifonos Dt 240 Pro 34 Ohms,"$2,949",,,https://articulo.mercadolibre.com.mx/MLM-67097...
7,Beyerdynamic Dt-990 Pro 250ohms Buen Fin Navid...,"$4,500",,,https://articulo.mercadolibre.com.mx/MLM-83432...
8,Beyerdynamic Headphones In-ear Beat Byrd Black...,$809,,,https://articulo.mercadolibre.com.mx/MLM-78967...
9,Beyerdynamic Aventho Wireless Black Audifonos ...,"$10,980",,,https://articulo.mercadolibre.com.mx/MLM-76969...


# Scrapping wallmart mx

In [2]:
import pandas as pd
import requests
import json
from pandas.io.json import json_normalize
import time

headers = {
    'Accept': 'application/json',
    'Referer': 'https://www.walmart.com.mx/productos?Ntt=xiaomi&page=0',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36',
    'Content-Type': 'application/json',
    'Cookie': 'Cookie_1=value; TS014f2e15=01538efd7c027ce77e369e2bc6d77f0399abd465a588ddf5af91f7df97ebd11d0588faf61dd6c5ce95485742c700a31dd976d66ee4; akavpau_vp_walmart_ss=1634878568~id=8dbbff347a90a0bea31a88fa333da0d1'
  }

prods = pd.DataFrame([])

for page in range(1,5):

  url = f"https://www.walmart.com.mx/api/v2/page/search?Ntt=xiaomi&page={page}&size=48"
  r = requests.request("GET", url, headers=headers)
  data = json.loads(r.text)
  prods = prods.append(json_normalize(data['appendix']['SearchResults']['content']), ignore_index=True)
  time.sleep(4)
  print(f'Getting page {page} ...')


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead


Getting page 1 ...
Getting page 2 ...
Getting page 3 ...
Getting page 4 ...


In [3]:
print(len(prods))

192


In [5]:
prods.head()


,id,productId,skuDisplayName,productSeoUrl,productRatings,weighable,freeShippingItem,skuPrice,bigItem,onDemand,bundle,preOrderable,pickUpEnabled,brandName,score,salesRank,variants,imageUrls.small,imageUrls.large,specialEventName,specialEventImage,unitOfMeasure,onSale
0,00750230095439,00750230095439,Smartphone Redmi Note 9 4G 4Gb 128Gb Negro Xia...,/celulares/smartphones/celulares-desbloqueados...,0.0,False,No Free Shipping,5299.0,False,False,False,False,True,xiaomi,12.307381,0.000000e+00,[],/gm/3pp/asr/2fb44d2b-7e71-4272-9d84-9aba4f514d...,/gm/3pp/asr/2fb44d2b-7e71-4272-9d84-9aba4f514d...,NaN,NaN,NaN,NaN
1,00046990294261,00046990294261,Smartphone Xiaomi Redmi 9T 128Gb 4Gb CARBON GR...,/celulares/smartphones/celulares-desbloqueados...,0.0,False,No Free Shipping,4950.0,False,False,False,False,True,xiaomi,12.307381,8.106390e-07,[],/gm/3pp/asr/bf20f6b8-66a3-4d6b-8dff-5f5e7d207a...,/gm/3pp/asr/bf20f6b8-66a3-4d6b-8dff-5f5e7d207a...,NaN,NaN,NaN,NaN
2,00693417772677,00693417772677,Xiaomi bocina Portable Bluetooth Speaker GRI...,/audio/bocinas-y-bafles/bocinas/xiaomi-bocina-...,0.0,False,No Free Shipping,579.0,False,False,False,False,True,xiaomi,12.307381,4.382517e-07,[],/gm/3pp/asr/36c1ec19-fa36-4e1b-9f44-aeeaa91fff...,/gm/3pp/asr/36c1ec19-fa36-4e1b-9f44-aeeaa91fff...,Envío en 48 horas,https://i5.walmartimages.com/dfw/fc2bdac6-b013...,NaN,NaN
3,00019099700077,00019099700077,Mi Truck Builder | Xiaomi | Robot Juguete de b...,/d/f/l/mi-truck-builder-xiaomi-robot-juguete-d...,0.0,False,No Free Shipping,499.0,False,False,False,False,True,xiaomi,12.307381,7.505917e-08,[],/gm/3pp/asr/33368528-14b9-453c-9c6c-c7183d7e5a...,/gm/3pp/asr/33368528-14b9-453c-9c6c-c7183d7e5a...,NaN,NaN,NaN,NaN
4,00890612903158,00890612903158,Xiaomi Redmi Note 10s 128 GB 6 GB RAM 64 MP Du...,/celulares/smartphones/celulares-desbloqueados...,0.0,False,No Free Shipping,6198.0,False,False,False,False,True,xiaomi,12.307381,0.000000e+00,[],/gm/3pp/asr/42be58b2-68ca-4451-b304-6f1c24e90f...,/gm/3pp/asr/42be58b2-68ca-4451-b304-6f1c24e90f...,NaN,NaN,NaN,NaN


# scrapping sams

In [8]:
def get_url(search_term):
  link = 'https://www.sams.com.mx/search/Ntt={}'.format(search_term.replace(' ','-'))
  return link

In [9]:
url = get_url('pantalla')
wd.get(url)
soup = BeautifulSoup(wd.page_source, 'html.parser')

In [15]:
results = soup.find_all('div', {'class': 'itemBox-container-wrp grid-itemBox-wrp first newAtc-itemBox-container-wrp'})

In [16]:
print(len(results))

0


In [17]:
print(soup)

<html class="no-touch samsProgressBar-busy" id="touchId" lang="es"><head><style type="text/css">@charset "UTF-8";[ng\:cloak],[ng-cloak],[data-ng-cloak],[x-ng-cloak],.ng-cloak,.x-ng-cloak,.ng-hide:not(.ng-hide-animate){display:none !important;}ng\:form{display:block;}.ng-animate-shim{visibility:hidden;}.ng-anchor{position:absolute;}</style>
<base href="/"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/><link href="//maps.googleapis.com" rel="dns-prefetch"/><link href="//www.googletagservices.com" rel="dns-prefetch"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta charset="utf-8"/>
<meta content="vnsMCl2gQX-YsbBSm2p8nE8vR5e0BuBH4JJ7_-AakcI" name="google-site-verification"/>
<meta id="seo-container-s"/>
<meta id="seo-container-f"/>
<title>Compra pantalla: Resultados de búsqueda</title>
<meta content="Compra pantalla: Resultados de búsqueda" itemprop="name"/>
<meta con

In [ ]:
'''
https://www.amazon.com.mx/Crawler-Control-Electric-Waterproof-Included/dp/B087NXVYF2/ref=pd_day0_2/146-0449573-7191119?pd_rd_w=5jHPH&pf_rd_p=1c8f9346-a87e-48c7-994a-82c93e96af91&pf_rd_r=76AYTQ5FQTV6VCEBHE6A&pd_rd_r=3fdccf3c-3e6a-44ad-b4a6-b96105683dec&pd_rd_wg=HNu3a&pd_rd_i=B087NXVYF2&psc=1
https://github.com/NVIDIA-AI-IOT/jetracer#cars
https://gsurma.medium.com/jetson-self-driving-toy-car-part-1-4341c139c0f2

'''